<a href="https://colab.research.google.com/github/EN10/BabyLlama/blob/main/Baby_Llama_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# training and inference code
!git clone https://github.com/karpathy/llama2.c.git
# tokenizer library
!pip install sentencepiece

Cloning into 'llama2.c'...
remote: Enumerating objects: 1507, done.
remote: Counting objects: 100% (573/573), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1507 (delta 535), reused 517 (delta 503), pack-reused 934
Receiving objects: 100% (1507/1507), 1.20 MiB | 15.00 MiB/s, done.
Resolving deltas: 100% (926/926), done.


In [ ]:
# download training data
!cd llama2.c && python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:38<00:00, 41.5MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [ ]:
# remove byte fallback <0x00> ... <0xFF> on line 113
!sed -i "113s/.*/                                   byte_fallback=False,/" ./llama2.c/tinystories.py
# change bfloat16 to float16 on line 72
!sed -i "72s/.*/dtype = 'float16'  # float32|bfloat16|float16/" ./llama2.c/train.py

In [ ]:
# create vocab
#!cd llama2.c && python tinystories.py train_vocab --vocab_size=105
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.model
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/raw/main/tok105/tok105.vocab

# use vocab to pretokenize training data
#!cd llama2.c && python tinystories.py pretokenize --vocab_size=105
#!cd llama2.c/data && tar -czvf tok105.tar.gz tok105
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.tar.gz
!cd ./llama2.c/data && tar -xf tok105.tar.gz

In [ ]:
# train neural net
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=105 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=1 --max_iters=1 #--init_from='resume'

In [ ]:
# compile run / inference executable
!cd llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok105.model

In [ ]:
# inference
!cd ./llama2.c && ./run out/model.bin -z data/tok105.bin -t 0.8 -n 256 -i "Once upon a time "